<a href="https://colab.research.google.com/github/moosunny/Transformer_Translator/blob/main/Seq2Seq_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
import pandas as pd
import numpy as np

# 한국어-영어 대화체 데이터 다운로드(link:)
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Transformer_Translation/2_대화체.xlsx')
data

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [78]:
data_kor = data["원문"]
data_eng = data["번역문"]

data_kor_list = list(data_kor)
data_eng_list = list(data_eng)

In [93]:
data_kor_list

['이번 신제품 출시에 대한 시장의 반응은 어떤가요?',
 '판매량이 지난번 제품보다 빠르게 늘고 있습니다.',
 '그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.',
 '네, 제가 연락해서 주문량을 2배로 늘리겠습니다.',
 '지난 회의 마지막에 논의했던 안건을 다시 볼까요?',
 '그보다는 이번 주 새로운 주제가 더 급한 것 같습니다.',
 '그럼 새로운 안건으로 회의를 시작하도록 하죠.',
 '네, 자료는 여러분의 앞에 미리 준비되어 있습니다.',
 '이번 주 금요일까지 2천개를 더 주문하라는 건가요?',
 '네, 시간이 조금 촉박하기는 하지만 가능해 보이는데요.',
 '주문은 가능하지만, 수령은 2달 정도 걸릴 것 같네요.',
 '이런, 저희는 물건을 최대한 빠르게 받고 싶어요.',
 '사무용품이나 가구 중 가장 먼저 바꿔야 할 것은 뭐라고 생각하나요?',
 '아무래도 컴퓨터가 아닐까요? 가장 많이 쓰니까요.',
 '저는 모두가 같이 쓰는 회의실 책상을 먼저 바꿔야 한다고 생각해요.',
 '그럴 수도 있겠네요. 회사에서 제일 오래되기도 했죠.',
 '오늘이나 내일 중에 본사에 가시는 분이 있나요?',
 '제가 오늘 오후 5시쯤 본사에 가려고 했습니다.',
 '그럼 오늘 회의 결과도 함께 가지고 가 주시겠어요?',
 '너무 늦지 않을까요? 지금 본사로 보내는 것이 더 빠를 것 같아요.',
 '행사 진행 아이디어가 있으면 지금 내주세요.',
 '모든 직원에게 메일로 아이디어를 2개씩 받으면 어떨까요?',
 '좋은 생각이네요! 지금 당장 메일을 보내야겠어요.',
 '네, 여러 명의 아이디어가 있으면 결과도 더 좋겠죠.',
 '오후에는 회의 대신 교육프로그램이 예정되어있지 않았나요?',
 '어제 교육프로그램이 연기되었다는 메일을 받지 못했나요?',
 '확인할 기회가 없었어요. 언제로 연기됐는지 아시나요?',
 '아마 다음 주 화요일 일 거에요. 다시 확인해볼게요.',
 '우리 제품은 어떤 방식으로 도쿄에 보내지나요?',
 '먼저 배로 도쿄 근처 항구까지 운

In [79]:
from transformers import AutoTokenizer

# 한국어 토크나이저 활용(upstage)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
vocab_size = len(tokenizer.get_vocab().items()) # tokenizer vocab_size

# 패딩을 오른쪽으로 설정
tokenizer.padding_side = "right"

data_kor_enc = tokenizer(data_kor_list, padding="max_length", max_length=50, truncation= True, return_tensors="pt")
data_eng_enc = tokenizer(data_eng_list, padding="max_length", max_length=50, truncation= True, return_tensors="pt")

loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--upstage--solar-1-mini-tokenizer/snapshots/7275739e7bbf66e4c9b308a4652bbce0a1f476dd/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--upstage--solar-1-mini-tokenizer/snapshots/7275739e7bbf66e4c9b308a4652bbce0a1f476dd/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [132]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

max_len = 50

# seed 고정
def seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed(42)

class CustomDataset(Dataset):
    def __init__(self, text, tokenizer, vocab_size, max_length = max_len, embedding_dim=1000):
        self.text = text
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
      text = self.text[idx]

      encoding = self.tokenizer(
                                text,
                                padding="max_length",
                                max_length=self.max_length,
                                truncation= True,
                                return_tensors="pt"
                                )
      input_idx = encoding["input_ids"]
      embedded_inputs = self.emb(input_idx)


      return embedded_inputs.squeeze(0) # (1, max_length, emb_dim)


# 토크나이저 로드 (BERT 기반)
tokenizer = AutoTokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")
vocab_size = len(tokenizer.get_vocab().items()) # 토큰 개수(solar-mini1 = 64000개)

# source 데이터 분리
train_src = data_kor_list[:89999]
val_src = data_kor_list[90000:94999]
test_src = data_kor_list[9500:]

# target 데이터 분리
train_target = data_eng_list[:89999]
val_target = data_eng_list[90000:94999]
test_target = data_eng_list[95000:]


# 데이터셋 생성
train_src_dataset = CustomDataset(text = train_src, tokenizer = tokenizer,  vocab_size = vocab_size)
val_src_dataset = CustomDataset(text = val_src, tokenizer = tokenizer,  vocab_size = vocab_size)
test_src_dataset = CustomDataset(text = test_src, tokenizer = tokenizer,  vocab_size = vocab_size)

# train dataloader
train_src_dataloader = DataLoader(train_src_dataset, batch_size=128, shuffle=False)
val_src_dataloader = DataLoader(val_src_dataset, batch_size=128, shuffle=False)
test_src_dataloader = DataLoader(test_src_dataset, batch_size=128, shuffle=False)

loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--upstage--solar-1-mini-tokenizer/snapshots/7275739e7bbf66e4c9b308a4652bbce0a1f476dd/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--upstage--solar-1-mini-tokenizer/snapshots/7275739e7bbf66e4c9b308a4652bbce0a1f476dd/tokenizer_config.json
loading file chat_template.jinja from cache at None


In [133]:
# target data
train_target_dataset = CustomDataset(text = train_target, tokenizer = tokenizer,  vocab_size = vocab_size)
val_target_dataset = CustomDataset(text = val_target, tokenizer = tokenizer,  vocab_size = vocab_size)
test_target_dataset = CustomDataset(text = test_target, tokenizer = tokenizer,  vocab_size = vocab_size)

train_target_dataloader = DataLoader(train_target_dataset, batch_size=128, shuffle=False)
val_target_dataloader = DataLoader(val_target_dataset, batch_size=128, shuffle=False)
test_target_dataloader = DataLoader(test_target_dataset, batch_size=128, shuffle=False)

In [141]:
import random

class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.lstm = nn.LSTM(input_dim, hidden_dim, dropout=dropout, batch_first = True)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src):
    embedded = self.dropout(src)
    output, (hidden, cell) = self.lstm(embedded)
    return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(hidden_dim, hidden_dim, n_layers, dropout=dropout, batch_first = True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)
        # input = [1, batch size]
        output, (hidden, cell) = self.rnn(input, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(1))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_length = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_length, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)

        # 첫 입력을 trg의 첫 번째 타임스텝으로 설정
        input = trg[:, 0, :]  # 이미 임베딩된 상태이므로 그대로 사용 (batch_size, embedding_dim)

        for t in range(1, trg_length):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output  # 저장 (batch_size, output_dim)
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)  # 예측된 단어 인덱스
            input = trg[:, t, :] if teacher_force else top1.unsqueeze(1).float()  # 차원 맞춰서 사용

        return outputs

In [145]:
max_len

NameError: name 'max_len' is not defined

In [146]:
input_dim = 50
output_dim = vocab_size
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


In [147]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(50, 512, batch_first=True, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (rnn): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=64000, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [148]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 38,189,568 trainable parameters


In [151]:
train_src_dataset[0]

tensor([[-0.9928, -0.2558, -0.9586,  ..., -0.8678, -0.1043,  0.9756],
        [-0.9928, -0.2558, -0.9586,  ..., -0.8678, -0.1043,  0.9756],
        [-0.9928, -0.2558, -0.9586,  ..., -0.8678, -0.1043,  0.9756],
        ...,
        [-1.8760,  0.4972, -0.2715,  ...,  0.8069, -0.2838,  1.3805],
        [-0.9585,  0.7996,  0.1341,  ...,  0.5574,  0.6178, -0.4554],
        [-0.0399, -0.2728, -0.8854,  ..., -0.1787,  0.9425,  0.0036]],
       grad_fn=<SqueezeBackward1>)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)